In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import gspread
from oauth2client.service_account import ServiceAccountCredentials


In [2]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('service_key.json', scope)
gc = gspread.authorize(credentials)

# Analytics

Based on twitter data scrap of mental health issues (PT-BR)

-- Data is not public

## Data from GCP

In [6]:
spreadsheet_key = '1hMzLN_Zt5sZ5rFA3CBJ0JlNDN4XhQauMiZqQmoGONeo'
book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheet("tweets")
table = worksheet.get_all_values()

In [80]:
##Convert table data into a dataframe
df = pd.DataFrame(table[1:], columns=table[0])

##Convert number strings to floats and ints
df = df.apply(pd.to_numeric, errors='ignore')

##Convert date strings to datetime format
df['created_at'] = pd.to_datetime(df['created_at'],infer_datetime_format=True)


In [81]:
df.shape

(2832, 12)

In [73]:
#train = df[df.y.notna()]
#test = df[df.y.isna()]

## Analytics

### Preprocessing

In [90]:
#!pip install tweet-preprocessor

In [110]:
import preprocessor as tp
import re
from unidecode import unidecode
#important libraries for preprocessing using NLTK
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
#nltk.download
#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('omw-1.4')
#nltk.download('punkt')
from nltk import tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amand\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [82]:
#remove accents
df['text'] = df['text'].apply(lambda x : unidecode(x))

In [83]:
#remove ponctuations and lower case
df['text'] = df['text'].apply(lambda x : re.sub(r'[^\w\s]', '', x).lower())

In [94]:
#deals with urls, mentions, reserved words (RT, FAV), *emojis, *smileys - [*] what im using
df['text'] = df['text'].apply(lambda x : tp.clean(x))

In [100]:
#remove stop words
def remove_stopwords(text):
    tokens = text.split()
    return ' '.join([word for word in tokens if word not in stopwords.words('portuguese')])

df['text'] = df['text'].apply(lambda x : remove_stopwords(x))

In [112]:
#tokenize
df['text'] = df['text'].apply(lambda x : tokenize.word_tokenize(x, language = 'portuguese'))

In [113]:
df.head()

,,author_id,context_annotations,created_at,edit_history_tweet_ids,entities,geo,id,lang,source,text,y
0,0,1486803274338643977,"[{'domain': {'id': '46', 'name': 'Business Tax...",2022-11-26 00:51:56+00:00,[1596305720626262016],,,1596305720626262016,pt,Twitter for Android,"[krlh, anime, romance, deixa, depressao, poha]",0.0
1,1,1438560931399835651,,2022-11-26 00:51:52+00:00,[1596305703991271424],"{'annotations': [{'start': 0, 'end': 6, 'proba...",,1596305703991271424,pt,Twitter for Android,"[cafalho, so, depressao, nessa, desgraca, vai,...",1.0
2,2,1318048828797444098,,2022-11-26 00:51:52+00:00,[1596305703433809920],,,1596305703433809920,pt,Twitter for Android,"[jimmysea, oferecendo, apenas, amor, incondici...",0.0
3,3,1563947975923073025,"[{'domain': {'id': '46', 'name': 'Business Tax...",2022-11-26 00:51:52+00:00,[1596305703248986113],"{'annotations': [{'start': 56, 'end': 65, 'pro...",,1596305703248986113,pt,Twitter for iPhone,"[saber, talvez, vou, morrer, nunca, ir, show, ...",0.0
4,4,1579194640380592130,,2022-11-26 00:51:51+00:00,[1596305698685480960],,,1596305698685480960,pt,Twitter for iPhone,"[cupido, deve, ter, cometido, suicidio, nn, po...",0.0
